In [1]:
%%time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import os
from REINFORCE_soft import MultiStockEnv, Agent, backtest, maybe_make_dir
import market_tickers
from model_config import model_parameters

Wall time: 2.27 s


In [2]:
# papermill parameters
SEED_LIST = [0] # only works for one value as it is now

TICKERS = market_tickers.NIK_25_TICKER
#TICKERS = ['AAPL', 'AMD', 'CSCO', 'F', 'INTC']
MARKET_NAME = 'NIK_25'
MODEL_BASE_NAME = 'RL_all_inp' # use either | 'RL_CNN' | 'RL_str_fcast' | 'RL_all_inp' |
FROM = '2018-01-01' # start of backtest
UNTIL = '2018-06-01' # end of backtest
NB_EPISODES = 200 # backtest model that was trained for this number of episodes

In [3]:
# other constants and parameters
TICKERS.sort()
FILE_PERIOD = '1d' # weekly='5d', daily='1d'
HALF_SPREAD = 0.0005/2.0 # 'a' in transaction cost function
NONLIN_COEFF = 1.0 # 'b' transaction cost function
POWER = 1.5 # power for change in poertfolio vector used in transaction cost
INIT_PORTFOLIO = 100000000.0 # initial portfolio value

USE_FORECASTS = model_parameters[MODEL_BASE_NAME]['use_forecasts'] # whether to use forecasts as inputs to agent policy net or not
NB_FORECASTS = model_parameters[MODEL_BASE_NAME]['nb_forecasts'] # number of forecasts to use in state. Set to None for no forecasts
FORECAST_TYPE = model_parameters[MODEL_BASE_NAME]['forecast_type'] # use 'strong' or 'weak' forecasts (if no forecasts given, this does not matter)
USE_CNN_STATE = model_parameters[MODEL_BASE_NAME]['use_cnn_state'] # whether to pass log-rets block for CNN part of policy net (set to false for forecast only)

TEST_NAME = f'{MARKET_NAME}_{MODEL_BASE_NAME}_seeds_{SEED_LIST[0]}_etc'
RESULTS_DIR = f'experiments/{MARKET_NAME}/seeded/{MODEL_BASE_NAME}/backtests/'
DATA_DIR = f'../../data/{MARKET_NAME}/preprocessed_data/'

In [4]:
frontier_df = pd.read_csv(f'{RESULTS_DIR}{MARKET_NAME}_{MODEL_BASE_NAME}_seed_frontiers.csv')
frontier_df = frontier_df[frontier_df['seed']==SEED_LIST[0]]
frontier_df.head()

,gamma_trade,gamma_risk,seed,excess_return,excess_risk,sharpe,is_pareto
0,45.0,18.0,0,-0.022548,0.660421,-0.034141,True
1,60.0,18.0,0,0.063777,0.675021,0.094482,True
2,30.0,32.0,0,0.086812,0.680020,0.127662,True
3,10.0,100.0,0,0.090404,0.690127,0.130996,True
4,20.0,56.0,0,0.128010,0.692018,0.184981,True


In [5]:
def make_int(x):
    if x.is_integer():
        x = int(x)
    return x

In [6]:
gamma_trade_list = [make_int(i) for i in list(frontier_df['gamma_trade'].round(3))]
gamma_risk_list = [make_int(j) for j in list(frontier_df['gamma_risk'].round(3))]

GAMMA_PAIRS = list(zip(gamma_trade_list, gamma_risk_list))
#GAMMA_PAIRS

######### remove later ##########
# this is just to test a small set
GAMMA_PAIRS = GAMMA_PAIRS[3:5]
GAMMA_PAIRS
#################################

[(10, 100), (20, 56)]

In [7]:
print('number of pairs: ', len(GAMMA_PAIRS))
#GAMMA_PAIRS

number of pairs:  2


## create agent(s) and backtest in new backtest env

In [8]:
%%time
# make directory for results if it does not exist already
maybe_make_dir(RESULTS_DIR)

print('creating environment and agent...')
env = MultiStockEnv(tickers=TICKERS, 
                    from_date=FROM,
                    until=UNTIL, 
                    #nb_episodes=100,
                    cash_key='USDOLLAR', 
                    #gamma_risk=GAMMA_RISK, 
                    #gamma_trade=GAMMA_TRADE,
                    #half_spread=HALF_SPREAD, 
                    nonlin_coef=NONLIN_COEFF, 
                    power=POWER, 
                    datadir=DATA_DIR, 
                    state_lookback_window=20, 
                    #distribution=EPISODE_DRAW_DISTRIBUTION,
                    #days_duration=DAYS_IN_EPISODE, 
                    mode='backtest', 
                    random_seed=7,
                    init_portfolio=INIT_PORTFOLIO, 
                    period_in_file_name=FILE_PERIOD, 
                    nb_forecasts=NB_FORECASTS, 
                    forecast_type=FORECAST_TYPE,
                    use_CNN_state=USE_CNN_STATE,
                    verbose=False)

agent = Agent(alpha=0.001, 
              gamma=0.99, 
              n_assets=len(TICKERS)+1, 
              tau=5, 
              lookback_window=20, 
              n_feature_maps=len(TICKERS)+1,
              use_forecasts=USE_FORECASTS,
              use_CNN_state=USE_CNN_STATE)

results_dict = {}
#actions_dict = {}
print('backtesting:')

total_backtests = len(SEED_LIST) * len(GAMMA_PAIRS)
backtest_counter = 0

for RANDOM_SEED in SEED_LIST:
    for (GAMMA_TRADE, GAMMA_RISK) in GAMMA_PAIRS:
        
        backtest_counter += 1
        print(f'\t[{backtest_counter}/{total_backtests}]\t{MODEL_BASE_NAME}: gamma_trade={GAMMA_TRADE}, gamma_risk={GAMMA_RISK}, seed={RANDOM_SEED}...')
        
        # load trained RL model for backtesting
        SAVED_MODEL_DIR = f'experiments/{MARKET_NAME}/seeded/{MODEL_BASE_NAME}/seed_{RANDOM_SEED}/trade_{GAMMA_TRADE}_risk_{GAMMA_RISK}/models/{MODEL_BASE_NAME}_seed_{RANDOM_SEED}_{NB_EPISODES}'
        # set dir where actions csv file will be saved
        actions_dir = f'experiments/{MARKET_NAME}/seeded/{MODEL_BASE_NAME}/seed_{RANDOM_SEED}/trade_{GAMMA_TRADE}_risk_{GAMMA_RISK}'
        
        try:
            rl_rets, rl_actions = backtest(agent, env, weights_file_dir=SAVED_MODEL_DIR, verbose=False)
            
            #rl_rets = pd.Series(data=rl_rets, index=env.returns.loc[FROM:UNTIL].iloc[1:].index)
            #rl_actions = pd.DataFrame(data=rl_actions, index=env.returns.loc[FROM:UNTIL].iloc[0:].index, columns=TICKERS+['USDOLLAR'])
            rl_rets = pd.Series(data=rl_rets, index=env.returns.loc[FROM:UNTIL].iloc[:-1].index)
            rl_rets = rl_rets.shift(1)
            rl_rets.iloc[0] = 0
            rl_actions = pd.DataFrame(data=rl_actions[:-1], index=env.returns.loc[FROM:UNTIL].iloc[:-1].index, columns=TICKERS+['USDOLLAR'])
            
            #print(f'rets: {rl_rets.shape}')
            #print(f'actions: {rl_actions.shape}')            
            
            # save actions
            rl_actions.to_csv(f'{actions_dir}/actions.csv')
            #print('actions saved.')

            cash_rets = env.returns.loc[FROM:UNTIL, env.cash_key].iloc[:-1]
            excess_rets = rl_rets - cash_rets

            R_e = excess_rets.mean() * 100 * 252
            sigma_e = excess_rets.std() * 100 * np.sqrt(252)
            sharpe = R_e / sigma_e

            results_dict[(GAMMA_TRADE, GAMMA_RISK, RANDOM_SEED)] = {'excess_return':R_e, 'excess_risk':sigma_e, 'sharpe':sharpe}
        except Exception as e:
            print(f'**error for model: trade_{GAMMA_TRADE}_risk_{GAMMA_RISK}')
            print(e)

creating environment and agent...
backtesting:
	[1/2]	RL_all_inp: gamma_trade=10, gamma_risk=100, seed=0...
	[2/2]	RL_all_inp: gamma_trade=20, gamma_risk=56, seed=0...
Wall time: 8.74 s


In [9]:
results_df = pd.DataFrame(results_dict).T.reset_index()
results_df.columns = ['gamma_trade', 'gamma_risk', 'seed', 'excess_return', 'excess_risk', 'sharpe']
#results_df.to_csv(RESULTS_DIR+TEST_NAME+'.csv', index=False)
#print('saved.')
results_df

,gamma_trade,gamma_risk,seed,excess_return,excess_risk,sharpe
0,10,100,0,0.111248,0.161198,0.690129
1,20,56,0,0.062408,0.172920,0.360906


In [12]:
rl_rets

Date
2018-01-01    0.000000
2018-01-04   -0.000195
2018-01-05    0.000175
2018-01-08    0.000084
2018-01-09    0.000056
                ...   
2018-05-25    0.000005
2018-05-28    0.000075
2018-05-29    0.000087
2018-05-30    0.000033
2018-05-31   -0.000056
Length: 106, dtype: float32

In [10]:
cash_rets

Date
2018-01-01    0.000055
2018-01-04    0.000056
2018-01-05    0.000055
2018-01-08    0.000057
2018-01-09    0.000057
                ...   
2018-05-25    0.000074
2018-05-28    0.000074
2018-05-29    0.000076
2018-05-30    0.000076
2018-05-31    0.000076
Name: USDOLLAR, Length: 106, dtype: float64

In [11]:
print(f'done backtesting for {MODEL_BASE_NAME}.')

done backtesting for RL_all_inp.
